# Import and initialization

In [13]:
import json
import os
import random
import requests
import numpy as np

from datetime import datetime

In [14]:
from checklist.test_suite import TestSuite
from checklist.test_types import MFT, INV, DIR

In [15]:
TYPES_AUG = ["keyboard_aug", "letter_aug", "letter_del"]

In [16]:
INTENTS = [
    'book-appointment',
    'book-hotel',
    'book-restaurant',
    'change-appointment',
    'change-car-rental',
    'change-driver',
    'change-hotel',
    'change-reminder',
    'change-restaurant',
    'contact-provider',
    'context',
    'find-Property',
    'find-Recipe',
    'find-Trip',
    'find-activity',
    'find-around-me',
    'find-car-rental',
    'find-driver',
    'find-flight',
    'find-hotel',
    'find-information',
    'find-itinerary',
    'find-restaurant',
    'find-train',
    'inform',
    'irrelevant',
    'other',
    'provide-agenda',
    'provide-news',
    'provide-showtimes',
    'provide-translation',
    'provide-tv-guide',
    'provide-weather',
    'purchase',
    'set-reminder',
    'smalltalk',
    'unsupported',
]

INTENTS_REDUCT = [
    'find-around-me',
    'find-flight',
    'find-hotel',
    'find-restaurant',
    'find-train',
    'irrelevant',
    'provide-showtimes',
    'purchase',
]

# Tests creation

In [33]:
def load_data(type_aug, data_path="data") :
    with open(os.path.join(data_path, f'dataset_{type_aug}.json'), "r") as f:
        data = json.load(f)

    random.shuffle(data)
    data_original = { "data": [elem.get("sentence") for elem in data],
                      "labels": [INTENTS.index(elem.get("intent")) for elem in data]}
    
    data_aug = { "data": [elem.get(type_aug) for elem in data],
                  "labels": [INTENTS.index(elem.get("intent")) for elem in data]}
    return(data_original, data_aug)


In [34]:
def create_tests(type_aug):
    data_original, data_aug = load_data(type_aug=type_aug)

    # MFT tests
    test_MFT_original = MFT(**data_original, name=f"MFT original {type_aug}", capability=type_aug, description="")
    test_MFT_aug = MFT(**data_aug, name=f"MFT {type_aug}", capability=type_aug, description="")

    # INV test
    sentences_original = data_original.get("data")
    sentences_aug = data_aug.get("data")
    t = { "data": [[sentences_original[i], sentences_aug[i]] for i in range(len(sentences_original))]}
    test_INV_aug = INV(**t, name=f"INV {type_aug}", capability=type_aug, description="")

    suite.add(test_MFT_original, overwrite=True)
    suite.add(test_MFT_aug, overwrite=True)
    suite.add(test_INV_aug, overwrite=True)

In [35]:
suite = TestSuite()

In [36]:
for type_aug in TYPES_AUG :
    create_tests(type_aug=type_aug)

# Tests running

In [37]:
def get_classif(classification):
    value = INTENTS.index(max(classification, key=classification.get))
    if value not in [15, 18, 19, 22, 23, 25, 29, 33] :
        value = 25
    return value

def get_softmax(classification):
    return [c/100 for c in classification.values()]


def get_classification(sentences):
    URL = "http://teaching-pfe.francecentral.azurecontainer.io/api/classification"  
    LANG = "fr-FR"
    classifs = [requests.get(f"{URL}?iso_language={LANG}&sentence={sentence}").json().get("classification") for sentence in sentences]
    return (np.array([get_classif(c) for c in classifs]), np.array([get_softmax(c) for c in classifs]))

In [38]:
start = datetime.now()
print(f"Started at {start} \n")

suite.run(get_classification, overwrite=True)

end = datetime.now()
print(f"\nExecution time : {end - start}")

Started at 2021-01-08 23:56:24.498857 

Running MFT original keyboard_aug
Predicting 200 examples
Running MFT keyboard_aug
Predicting 200 examples
Running INV keyboard_aug
Predicting 400 examples
Running MFT original letter_aug
Predicting 200 examples
Running MFT letter_aug
Predicting 200 examples
Running INV letter_aug
Predicting 400 examples
Running MFT original letter_del
Predicting 200 examples
Running MFT letter_del
Predicting 200 examples
Running INV letter_del
Predicting 400 examples

Execution time : 0:30:44.478617


In [39]:
suite.summary()

letter_del

MFT original letter_del
Test cases:      200
Fails (rate):    0 (0.0%)


MFT letter_del
Test cases:      200
Fails (rate):    4 (2.0%)

Example fails:
25 (0.0) À quelle heure le premier trai arrivera-t-il demain matin à nantes ?
----
25 (0.0) passe comande sur amazon
----
25 (0.0) Coucou mon génie, tu as un esto a me conseiller sur oulan bator ?
----


INV letter_del
Test cases:      200
Fails (rate):    3 (1.5%)

Example fails:
23 (1.0) À quelle heure le premier train arrivera-t-il demain matin à nantes ?
25 (0.0) À quelle heure le premier trai arrivera-t-il demain matin à nantes ?

----
22 (1.0) Coucou mon génie, tu as un resto a me conseiller sur oulan bator ?
25 (0.0) Coucou mon génie, tu as un esto a me conseiller sur oulan bator ?

----
22 (1.0) Conseille moi un restaurant tibétain
25 (0.0) Conseille moi un retaurant tibétain

----




keyboard_aug

MFT original keyboard_aug
Test cases:      200
Fails (rate):    1 (0.5%)

Example fails:
25 (0.4) j'ai besoin que tu me 

# Saving tests suite

In [40]:
if not os.path.isdir("testsuite") :
    os.mkdir("testsuite")
suite.save(os.path.join("testsuite", "suite.pkl"))